# <a href=https://github.com/fuodorov/REDPIC>REDPIC</a> vs ASTRA

<a href='https://fuodorov.github.io'>V. Fedorov</a>

In [1]:
import redpic as rp
import kenv as kv
import holoviews as hv
import numpy as np 
import pandas as pd
import glob

hv.notebook_extension('matplotlib')

%output size=250 backend='matplotlib' fig='png' dpi=200

%opts Curve [show_grid=True aspect=5] (linewidth=1 alpha=0.7 color='blue')
%opts Scatter.redpic [show_grid=True aspect=5] (alpha=0.4 s=0.3 color='red')
%opts Scatter.astra [show_grid=True aspect=5] (alpha=0.4 s=0.3 color='blue')

/Users/fuodorov/Documents/GitHub/redpic/redpic/solver.py:57: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "sum_field_particles" failed type inference due to: No conversion from tuple(array(float64, 1d, C) x 3) to array(float32, 1d, A) for '$418.5', defined at None

File "redpic/solver.py", line 76:
def sum_field_particles(x: np.array, y: np.array, z: np.array,
    <source elided>
                    Fz[i] += (z[i]-z[j]) / ((x[j] - x[i])**2 + (y[j] - y[i])**2 + (z[j] - z[i])**2)**(3/2)
    return Fx, Fy, Fz
    ^

[1] During: typing of assignment at /Users/fuodorov/Documents/GitHub/redpic/redpic/solver.py (76)

File "redpic/solver.py", line 76:
def sum_field_particles(x: np.array, y: np.array, z: np.array,
    <source elided>
                    Fz[i] += (z[i]-z[j]) / ((x[j] - x[i])**2 + (y[j] - y[i])**2 + (z[j] - z[i])**2)**(3/2)
    return Fx, Fy, Fz
    ^

  @vectorize([float32[:](float32[:], float32[:], float32[:], float32, f

NotImplementedError: array(float32, 1d, A) cannot be represented as a Numpy dtype

In [ ]:
rp.__version__

## Define the simulation

Define accelerator beamline parameters:

In [ ]:
acc = rp.Accelerator(z_start=0.7, z_stop=5, dz=0.01)

In [ ]:
#              Unique name,  z-position [m],  Ez [MV/m],  Ez(z) profile
acc.add_accel('Acc. 1',      4.096,          -1.1,         'Ez.dat')
acc.add_accel('Acc. 2',      5.944,          -1.1,         'Ez.dat')

In [ ]:
#                 Unique name,  z-position [m],  Bz [T],  Bz(z) profile
acc.add_solenoid('Sol. 1',      0.450,           -0.058,   'Bz.dat')
acc.add_solenoid('Sol. 2',      0.957,           0.0390,   'Bz.dat')
acc.add_solenoid('Sol. 3',      2.107,           0.0250,   'Bz.dat')
acc.add_solenoid('Sol. 4',      2.907,           0.0440,   'Bz.dat')
acc.add_solenoid('Sol. 5',      3.670,           0.0400,   'Bz.dat')
acc.add_solenoid('Sol. 6',      4.570,           0.0595,   'Bz.dat')
acc.add_solenoid('Sol. 7',      5.470,           0.0590,   'Bz.dat')

In [ ]:
acc.compile()

Define the electron beam parameters:

In [ ]:
beam = rp.Beam(rp.electron, charge=-1.0e-5) # ON space charge

In [ ]:
beam.upload('Example.ini')

In [ ]:
sim = rp.Simulation(beam, acc)

In [ ]:
def bench(n_iter=1):
    for iter in range(n_iter):
        sim.track()

In [ ]:
%load_ext line_profiler

In [ ]:
%lprun -f sim.track bench() 

## Compare to ASTRA

In [ ]:
track_files = np.sort(glob.glob('*.*[0-9].csv'))
cols = ['x', 'y', 'z', 'px', 'py', 'pz', 'Ex', 'Ey', 'Ez', 'Bx', 'By', 'Bz']
i_to_plot = np.arange(0, len(track_files), 1)

In [ ]:
def read_track_red(fname):
    df = pd.read_csv(fname, dtype='float32')
    df['x'] = df['x']*1e3 # mm
    df['y'] = df['y']*1e3 # mm
    df['Bx'] = df['Bx']*1e4 # Gs
    df['By'] = df['By']*1e4 # Gs
    df['Bz'] = df['Bz']*1e4 # Gs
    #df = df.sample(n=10_0000) # if particles number > 100_000  
    return df

In [ ]:
def read_track_astra(fname):
    cols = ['x', 'y', 'z', 'px', 'py', 'pz', 'clock', 'charge', 'id', 'flag']
    #        m    m    m    eV/c  eV/c  eV/c  ns       nC
    df = pd.read_csv(fname, header=None, delim_whitespace=True, names=cols, dtype='float32')
    
    df = df[df.flag !=-15] # ignore the lost particles
    
    df['x'] = df['x']*1e3 # cm
    df['y'] = df['y']*1e3 # cm
    
    df['px'] = df['px']/1e6 # MeV/c
    df['py'] = df['py']/1e6 # MeV/c
    
    # remove the reference particle
    df0 = df.head(1)
    df  = df.drop(df0.index)
    
    z0  = df0.z.values[0]
    pz0 = df0.pz.values[0]
    
    # Recalculate z and pz:
    
    df['z'] = z0 + df['z'] # m
    df['pz'] = (pz0 + df['pz'])/1e6 # MeV/c
    
    # For large number of macro-particles leave only 100000:
    #df = df.sample(n=10_000)
    
    return df

In [ ]:
dim_x = hv.Dimension('x', unit='mm', range=(-100, 100))
dim_y = hv.Dimension('y', unit='mm')
dim_z = hv.Dimension('z', unit='m', range=(acc.z_start, acc.z_stop))
dim_px = hv.Dimension('px', unit='MeV/c', label='$p_x$')
dim_py = hv.Dimension('py', unit='MeV/c', label='$p_y$')
dim_pz = hv.Dimension('pz', unit='MeV/c', label='$p_z$')
dim_Ex = hv.Dimension('Ex', unit='MV/m', label='$E_x$')
dim_Ey = hv.Dimension('Ey', unit='MV/m', label='$E_y$')
dim_Ez = hv.Dimension('Ez', unit='MV/m', label='$E_z$')
dim_Bx = hv.Dimension('Bx', unit='Gs', label='$B_x$')
dim_By = hv.Dimension('By', unit='Gs', label='$B_y$')
dim_Bz = hv.Dimension('Bz', unit='Gs', label='$B_z$')

In [ ]:
def plot(i):
    red_df = read_track_red(track_files[i])
    astra_df = read_track_astra('Track.0263.001')
    
    red_z_x   = hv.Scatter(red_df, kdims=[dim_z, dim_x], group='redpic')
    astra_z_x  = hv.Scatter(astra_df, kdims=[dim_z,dim_x], group='astra')
    return (red_z_x*astra_z_x)

In [ ]:
items = [(i, plot(i)) for i in i_to_plot]

hv.HoloMap(items, kdims = ['Track file index']).collate()

## Technical info: